# WOfS Validation Data Clean  <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2),
[ga_ls8c_wofs_2_summary ](https://explorer.digitalearth.africa/ga_ls8c_wofs_2_summary),
[usgs_ls8c_level2_2]()

Notes:
* Landsat 8 collection 2 is confidential at continental level on 26 June 2020.
* This notebook should be run in Collection 2 Read Private Workspace should we need to run the Landsat 8 Collection 2 Sample dataset. 

## Background
The [Water Observations from Space (WOfS)](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs/about-wofs) is a derived product from Landsat 8 satellite observations as part of provisional Landsat 8 Collection 2 surface reflectance and shows surface water detected in Africa.
Individual water classified images are called Water Observation Feature Layers (WOFLs), and are created in a 1-to-1 relationship with the input satellite data. 
Hence there is one WOFL for each satellite dataset processed for the occurrence of water.

The data in a WOFL is stored as a bit field. This is a binary number, where each digit of the number is independantly set or not based on the presence (1) or absence (0) of a particular attribute (water, cloud, cloud shadow etc). In this way, the single decimal value associated to each pixel can provide information on a variety of features of that pixel. 
For more information on the structure of WOFLs and how to interact with them, see [Water Observations from Space](../Datasets/Water_Observations_from_Space.ipynb) and [Applying WOfS bitmasking](../Frequently_used_code/Applying_WOfS_bitmasking.ipynb) notebooks. 

## Description
This notebook explains how you can compile tables from Collect Earth Online tool from each partner institution and make them analysis-ready for accuracy assessment. 

The notebook demonstrates how to:

1. Load collected validation points as a list of observations each has a location and month
2. Data wrangling including cleaning the table, and mapping each point two twelve month observation 

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

After finishing the analysis, you can modify some values in the "Analysis parameters" cell and re-run the analysis to load WOFLs for a different location or time period.

### Load packages

In [122]:
%matplotlib inline

import datacube
from datacube.utils import masking, geometry 
import sys
import os
import dask 
import rasterio, rasterio.features
import xarray
import glob
import numpy as np
import pandas as pd
import seaborn as sn
import geopandas as gpd
import subprocess as sp
import matplotlib.pyplot as plt
import scipy, scipy.ndimage
import warnings
warnings.filterwarnings("ignore") #this will suppress the warnings for multiple UTM zones in your AOI 

sys.path.append("../Scripts")
from deafrica_plotting import display_map, rgb
from deafrica_spatialtools import xr_rasterize
from deafrica_datahandling import wofs_fuser, mostcommon_crs,load_ard
from rasterio.mask import mask

### Connect to the datacube

In [123]:
dc = datacube.Datacube()

### Analysis parameters

In [124]:
#make sure that validation points have at least three columns : location (x,y), class, as well as 12 records for each observation  
#Path to the validation data points csv file 
CEO = '../Supplementary_data/Validation/CEO_1_RCMRD_2020-07-30.csv'

### Loading Dataset

In [125]:
#Read in the validation data csv
df = pd.read_csv(CEO, delimiter=",")
ground_truth = df.drop(['SAMPLE_ID','USER_ID','IMAGERY_TITLE','COLLECTION_TIME','ANALYSIS_DURATION','PL_PLOTID'], axis=1)

In [126]:
ground_truth.columns

Index(['PLOT_ID', 'LON', 'LAT', 'FLAGGED', 'ANALYSES',
       'SENTINEL2MOSAICYEARMONTH',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, WATER WAS NOT OBSERVED?',
       'ENTER MONTHS[1-12] IN 2018, IMAGE WAS BAD?',
       'ENTER MONTHS[1-12] IN 2018, THAT YOU ARE UNSURE IF YOU OBSERVE WATER OR NOT? ',
       'WHAT IS THE FEATURE?', 'COMMENT'],
      dtype='object')

In [127]:
ground_truth.shape

(325, 12)

In [129]:
ground_truth = ground_truth.rename(columns={'WHAT IS THE FEATURE?':'CLASS','ENTER MONTHS[1-12] IN 2018, WATER WAS OBSERVED?':'WATER',
                                            'ENTER MONTHS[1-12] IN 2018, WATER WAS NOT OBSERVED?':'NO_WATER','ENTER MONTHS[1-12] IN 2018, IMAGE WAS BAD?':'BAD_IMAGE',
                                             'ENTER MONTHS[1-12] IN 2018, THAT YOU ARE UNSURE IF YOU OBSERVE WATER OR NOT? ':'NOT_SURE'})
ground_truth

,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2MOSAICYEARMONTH,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT
0,137387437,39.723471,-10.004402,False,1,2018 - 2018,1-12,0,0,0,Open water - freshwater,Na
1,137387438,39.674345,-10.082225,False,1,2018 - 2018,1-12,0,0,0,Open water - freshwater,Na
2,137387439,40.218464,-10.282358,False,1,2018 - 2018,1-12,0,0,0,Open water - freshwater,Na
3,137387440,34.575797,-10.328707,False,1,2018 - 2018,1-12,0,4,0,Open water - freshwater,Na
4,137387441,34.554654,-10.405651,False,1,2018 - 2018,1-12,0,4,0,Open water - freshwater,Na
...,...,...,...,...,...,...,...,...,...,...,...,...
320,137387757,44.916230,-25.221032,False,1,2018 - 2018,0,[1-12],0,0,Shrubs,Na
321,137387758,44.403204,-25.227514,False,1,2018 - 2018,0,[1-12],0,0,Barren (Bare soil / Rocky Land),Flood plain
322,137387759,46.168636,-25.243593,False,1,2018 - 2018,[1-12],0,0,0,Open water - marine,Na
323,137387760,44.938928,-25.434106,False,1,2018 - 2018,[1-12],0,1,0,Open water - marine,Na


In [130]:
#Converting all column types to string if not already
#ground_truth = ground_truth.astype(str)
ground_truth['NOT_SURE'] = ground_truth.NOT_SURE.astype(str)

In [131]:
cols = ['WATER','NO_WATER','BAD_IMAGE','NOT_SURE']
for col in cols:
    ground_truth[col] = ground_truth[col].str.replace('[','')
    ground_truth[col] = ground_truth[col].str.replace(']','')
    ground_truth[col] = ground_truth[col].str.replace('&','')
    ground_truth[col] = [''.join(c.split()) for c in ground_truth[col]]

In [132]:
#check whether any nan values in the dataframe and print it out against the column name 
# count_nan_in_df = ground_truth.isnull().sum()
# print (count_nan_in_df)

In [133]:
#replacing the name of months with their numerical values
replacements = { 'WATER': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'NO_WATER': {r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'},
               'BAD_IMAGE':{r'Jan':'1', r'Feb':'2',r'Mar':'3',r'Apr':'4',r'May':'5',r'Jun':'6',r'Jul':'7',r'Aug':'8',r'Sep':'9',r'Oct':'10',r'Nov':'11',r'Dec':'12'}}

ground_truth.replace(replacements, regex=True, inplace=True)

In [134]:
ground_truth['SENTINEL2MOSAICYEARMONTH'] = ground_truth['SENTINEL2MOSAICYEARMONTH'].str.replace('2019-2019','2018-2018')

In [135]:
def split_str(row, newtable):
#check each row for No-WATER info an update the water column 
    monthstr=row['NO_WATER']
    if monthstr!='0'and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='0'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow)
#check each row for water info 
    monthstr=row['WATER']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='1'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow)  # update index / ignore original index
#check each row for bad image 
    monthstr=row['BAD_IMAGE']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='2'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow) 
    monthstr=row['NOT_SURE']
    if monthstr!='0' and monthstr!='nan':
        monthlist=[[int(i) for i in s.split('-')] for s in monthstr.split(',')]
        for l in monthlist:
            if len(l)==1: l=[l[0],l[0]]
            for i in range(l[0], l[1]+1):
                newrow=row[['PLOT_ID','LON','LAT','FLAGGED','ANALYSES','WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS','COMMENT']]
                newrow['MONTH']=f'{i:02d}'
                newrow['WATERFLAG']='3'
                newrow["SENTINEL2YEAR"]='2018'
                newtable=newtable.append(newrow) 
                
    return newtable


In [136]:
#count_nan_in_df = ground_truth.isnull().sum()
#print (count_nan_in_df)

In [137]:
#for check on any issues 
ground_truth.to_csv('../Supplementary_data/Validation/Refined/CEO_2.csv')

In [138]:
#Making an empty dataframe
result = pd.DataFrame()

In [139]:
for irow in range(len(ground_truth)):
    result=split_str(ground_truth.iloc[irow], result)
    result.update(result)

In [140]:
result.shape
#result.loc[13]#this shows all the table 

(3990, 14)

In [141]:
indexNames = result[result.duplicated(['LAT', 'LON','MONTH'], keep=False)]
indexNames.shape

(192, 14)

In [142]:
result = result[['PLOT_ID', 'LON', 'LAT','FLAGGED','ANALYSES','SENTINEL2YEAR', 'WATER','NO_WATER','BAD_IMAGE','NOT_SURE','CLASS', 'COMMENT', 'MONTH','WATERFLAG']]

In [143]:
indexNames = result[result.duplicated(['LAT', 'LON','MONTH'], keep=False) & (result['WATERFLAG'] <= '1') & (result['NOT_SURE']!='0')].index
result.drop(indexNames , inplace=True)
result

,PLOT_ID,LON,LAT,FLAGGED,ANALYSES,SENTINEL2YEAR,WATER,NO_WATER,BAD_IMAGE,NOT_SURE,CLASS,COMMENT,MONTH,WATERFLAG
0,137387437.0,39.723471,-10.004402,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,Na,01,1
0,137387437.0,39.723471,-10.004402,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,Na,02,1
0,137387437.0,39.723471,-10.004402,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,Na,03,1
0,137387437.0,39.723471,-10.004402,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,Na,04,1
0,137387437.0,39.723471,-10.004402,0.0,1.0,2018,1-12,0,0,0,Open water - freshwater,Na,05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,137387761.0,45.526576,-25.572893,0.0,1.0,2018,1-12,0,0,0,Open water - marine,Na,08,1
324,137387761.0,45.526576,-25.572893,0.0,1.0,2018,1-12,0,0,0,Open water - marine,Na,09,1
324,137387761.0,45.526576,-25.572893,0.0,1.0,2018,1-12,0,0,0,Open water - marine,Na,10,1
324,137387761.0,45.526576,-25.572893,0.0,1.0,2018,1-12,0,0,0,Open water - marine,Na,11,1


the part that looks into fixing the rows with multiple values for months should go here:


In [144]:
#group by PLOT Id and then do the estimation of the row number for each plot number in column Month  
count = result.groupby(['PLOT_ID'])['MONTH'].count()
count.to_csv('../Supplementary_data/Validation/Refined/CEO_1_RCMRD_count.csv')

In [68]:
#save the dataframe as csv file 
result.to_csv('../Supplementary_data/Validation/Refined/CEO_1_RCMRD_2020-07-30.csv')

In [69]:
#joining dataframes together and extract one csv for each partner institution 
DF = glob.glob('../Supplementary_data/Validation/Refined/*_RCMRD_*.csv')
frame = []
for d in DF: 
    f = pd.read_csv(d,delimiter=",")
    frame.append(f)
out = pd.concat(frame)
out.to_csv('../Supplementary_data/Validation/Refined/CEO_RCMRD_2020-07-30.csv')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** January 2020

**Compatible datacube version:** 

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)

In [ ]:
#test the groundtruth with a 6933 EPSG as well (conversion) - how to reproject